In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
full_dataset = r"C:\Frank\UoT 2024-2025\MIE8888 Project\full_dataset_2.11.csv"
df = pd.read_csv(full_dataset, encoding="Windows-1252")

C:\Users\frank\AppData\Local\Temp\ipykernel_41544\1841645292.py:2: DtypeWarning: Columns (1,4,10,16,52,85,86,91) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(full_dataset, encoding="Windows-1252")


Grouping treatments for co-occurence analysis  
Define criteria:   
1. Same patient, within the same visit range in 3 months
2. Same patient, within the same total visiting range (first to last visit)

In [ ]:
# Not useful for the task, just have a look at the data
# Generate Procedure Groups for Each Patient's Visit Range
df['procedure_date'] = pd.to_datetime(df['procedure_date'])
df['first_visit'] = pd.to_datetime(df['first_visit'])
df['last_visit'] = pd.to_datetime(df['last_visit'])

# Group by patient_id and visit range, collecting all procedure codes
grouped = df.groupby(['patient_id', 'first_visit', 'last_visit'])['procedure_code_y'].apply(list).reset_index()
grouped.head()

,patient_id,first_visit,last_visit,procedure_code_y
0,1,1991-04-30,2013-04-30,"[27211, 62501, 67211, 99111, 1203, 1203, 11107..."
1,2,1989-05-31,2017-02-14,"[21223, 21222, 21223, 27211, 67211, 21222, 214..."
2,3,1990-01-30,2004-05-06,"[12101, 11303, 2142, 23321, 24, 24, 2142, 1202..."
3,4,1990-01-30,2024-01-03,"[23321, 72211, 2142, 2112, 43412, 11303, 2142,..."
4,5,1989-09-21,2003-11-17,"[72221, 71101, 2142, 1202, 11303, 23511, 13401..."


In [ ]:
# Code explaination
import pandas as pd
from datetime import timedelta


# Convert procedure_date to datetime format
df['procedure_date'] = pd.to_datetime(df['procedure_date'])

# Define the time window for grouping
time_window = timedelta(days=__)  #set the time window here

# Sort dataset by patient and date
df = df.sort_values(by=['patient_id', 'procedure_date'])

# Create a list to store grouped procedures
grouped_procedures = []

# Group by patient_id
# process one patient at a time
# patient_data contains all procedures for a single patient.
for patient, patient_data in df.groupby('patient_id'):

    patient_data = patient_data.sort_values(by='procedure_date') # chronologically ordered
    temp_group = []  # Stores procedure codes within the 3-month window
    start_date = None  # Track the start of the time window
    
    # process each procedure code for that patient
    for index, row in patient_data.iterrows():

        # If temp_group is empty, this is the first procedure for this patient.
        # Sets start_date to the procedure date.
        # Adds the first procedure_code_y to the temp_group.
        if not temp_group:
            start_date = row['procedure_date']  # Set start date for the group
            temp_group.append(row['procedure_code_y'])
        else:
            # Check if the procedure falls within the 3-month window, 
            # If this procedure happens within 3 months of start_date, it's added to temp_group.
            if row['procedure_date'] - start_date <= time_window:
                temp_group.append(row['procedure_code_y'])

            # If a procedure falls outside the 3-month window:
            # The current temp_group is saved in grouped_procedures.
            # A new group is started with the current procedure.
            # start_date is updated to the new procedure's date.
            else:
                # Store the grouped procedures and reset for a new group
                grouped_procedures.append({'patient_id': patient, 'procedure_codes': list(set(temp_group))})
                temp_group = [row['procedure_code_y']]
                start_date = row['procedure_date']
    
    # Store the last group for each patient
    if temp_group:
        grouped_procedures.append({'patient_id': patient, 'procedure_codes': list(set(temp_group))})

In [23]:
import pandas as pd
from datetime import timedelta


# Convert procedure_date to datetime format
df['procedure_date'] = pd.to_datetime(df['procedure_date'])

# Define the time window for grouping
time_window = timedelta(days=90)  # 3 months

# Sort dataset by patient and date
df = df.sort_values(by=['patient_id', 'procedure_date'])

# Create a list to store grouped procedures
grouped_procedures = []

# Group by patient_id
for patient, patient_data in df.groupby('patient_id'):
    patient_data = patient_data.sort_values(by='procedure_date')
    temp_group = []  # Stores procedure codes within the 3-month window
    start_date = None  # Track the start of the time window
    
    for index, row in patient_data.iterrows():
        if not temp_group:
            start_date = row['procedure_date']  # Set start date for the group
            temp_group.append(row['procedure_code_y'])
        else:
            # Check if the procedure falls within the 3-month window
            if row['procedure_date'] - start_date <= time_window:
                temp_group.append(row['procedure_code_y'])
            else:
                # Store the grouped procedures and reset for a new group
                grouped_procedures.append({'patient_id': patient, 'procedure_codes': list(set(temp_group))})
                temp_group = [row['procedure_code_y']]
                start_date = row['procedure_date']
    
    # Store the last group for each patient
    if temp_group:
        grouped_procedures.append({'patient_id': patient, 'procedure_codes': list(set(temp_group))})

# Convert to DataFrame
grouped_3m_df = pd.DataFrame(grouped_procedures)
grouped_3m_df

,patient_id,procedure_codes
0,1,"[2112, 33121, 12101, 21221, 99111, 2601, 27211..."
1,1,[1203]
2,1,[2142]
3,1,"[1203, 43413]"
4,1,"[2144, 1204, 12101, 43414]"
...,...,...
121024,13580,"[2144, 2112, 11107, 2601, 11117, 1102, 12112]"
121025,13583,"[1204, 2111]"
121026,13584,[94303]
121027,13586,"[99111, 27211, 23115, 1205, 2111]"


In [24]:
import pandas as pd
from datetime import timedelta


# Convert procedure_date to datetime format
df['procedure_date'] = pd.to_datetime(df['procedure_date'])

# Define the time window for grouping
time_window = timedelta(days=180)  # 6 months

# Sort dataset by patient and date
df = df.sort_values(by=['patient_id', 'procedure_date'])

# Create a list to store grouped procedures
grouped_procedures = []

# Group by patient_id
for patient, patient_data in df.groupby('patient_id'):
    patient_data = patient_data.sort_values(by='procedure_date')
    temp_group = []  # Stores procedure codes within the 3-month window
    start_date = None  # Track the start of the time window
    
    for index, row in patient_data.iterrows():
        if not temp_group:
            start_date = row['procedure_date']  # Set start date for the group
            temp_group.append(row['procedure_code_y'])
        else:
            # Check if the procedure falls within the 3-month window
            if row['procedure_date'] - start_date <= time_window:
                temp_group.append(row['procedure_code_y'])
            else:
                # Store the grouped procedures and reset for a new group
                grouped_procedures.append({'patient_id': patient, 'procedure_codes': list(set(temp_group))})
                temp_group = [row['procedure_code_y']]
                start_date = row['procedure_date']
    
    # Store the last group for each patient
    if temp_group:
        grouped_procedures.append({'patient_id': patient, 'procedure_codes': list(set(temp_group))})

# Convert to DataFrame
grouped_6m_df = pd.DataFrame(grouped_procedures)
grouped_6m_df

,patient_id,procedure_codes
0,1,"[2112, 33121, 12101, 21221, 99111, 2601, 27211..."
1,1,[1203]
2,1,"[1203, 43413, 2142]"
3,1,[43413]
4,1,"[2144, 1204, 12101, 43414]"
...,...,...
103110,13580,"[2144, 2112, 11107, 2601, 11117, 1102, 12112]"
103111,13583,"[1204, 2111]"
103112,13584,[94303]
103113,13586,"[99111, 27211, 23115, 1205, 2111]"


In [ ]:
# Now try if we remove the basic-treatmen
df['procedure_code_y'] = df['procedure_code_y'].astype(str)
df_non_basic = df[~df['procedure_code_y'].str.match(r'^\d{4}$|^1\d{4}$')]

In [38]:
# Now try the 180-day time window

import pandas as pd
from datetime import timedelta


# Convert procedure_date to datetime format
df_non_basic['procedure_date'] = pd.to_datetime(df_non_basic['procedure_date'])

# Define the time window for grouping
time_window = timedelta(days=90)  # 3 months

# Sort dataset by patient and date
df_non_basic = df_non_basic.sort_values(by=['patient_id', 'procedure_date'])

# Create a list to store grouped procedures
grouped_procedures = []

# Group by patient_id
for patient, patient_data in df_non_basic.groupby('patient_id'):
    patient_data = patient_data.sort_values(by='procedure_date')
    temp_group = []  # Stores procedure codes within the 3-month window
    start_date = None  # Track the start of the time window
    
    for index, row in patient_data.iterrows():
        if not temp_group:
            start_date = row['procedure_date']  # Set start date for the group
            temp_group.append(row['procedure_code_y'])
        else:
            # Check if the procedure falls within the 3-month window
            if row['procedure_date'] - start_date <= time_window:
                temp_group.append(row['procedure_code_y'])
            else:
                # Store the grouped procedures and reset for a new group
                grouped_procedures.append({'patient_id': patient, 'procedure_codes': list(set(temp_group))})
                temp_group = [row['procedure_code_y']]
                start_date = row['procedure_date']
    
    # Store the last group for each patient
    if temp_group:
        grouped_procedures.append({'patient_id': patient, 'procedure_codes': list(set(temp_group))})

# Convert to DataFrame
grouped_non_basic_df = pd.DataFrame(grouped_procedures)
grouped_non_basic_df

C:\Users\frank\AppData\Local\Temp\ipykernel_12964\1747137785.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_non_basic['procedure_date'] = pd.to_datetime(df_non_basic['procedure_date'])


,patient_id,procedure_codes
0,1,"[43412, 21212, 27211, 25711, 21221, 33121, 99111]"
1,1,[43413]
2,1,[43414]
3,1,"[43414, 42311]"
4,1,[43414]
...,...,...
47653,13575,"[23112, 23113, 23314, 23313, 23323]"
47654,13579,[80671]
47655,13584,[94303]
47656,13586,"[23115, 27211, 99111]"


In [44]:
# Define output file path
output_file = r"C:\Frank\UoT 2024-2025\MIE8888 Project\M11\2.11 result\grouped_procedures_non_basic_3_months.csv"
grouped_non_basic_df.to_csv(output_file, index=False)
output_file = r"C:\Frank\UoT 2024-2025\MIE8888 Project\M11\2.11 result\grouped_procedures_6_months.csv"
grouped_6m_df.to_csv(output_file, index=False)
output_file = r"C:\Frank\UoT 2024-2025\MIE8888 Project\M11\2.11 result\grouped_procedures_3_months.csv"
grouped_3m_df.to_csv(output_file, index=False)